This is a test notebook to produce an image from HLS using s3 urls from the LP DAAC. 

In [ ]:
pip install leafmap pystac_client

Import the python libraries required for running the script

In [ ]:
import json
import leafmap
import stackstac
import xarray as xr

from pystac_client import Client


Initialize map to allow selection of desired AOI for study. Start with Pakistan for flood Visualization.

In [ ]:
map = leafmap.Map(center=(31.5, 22.5), zoom=8, 
                draw_control=True, measure_control=False, fullscreen_control=False, attribution_control=True)

map

In [ ]:
# Save the selected location as geojson
map.save_draw_features("bbox.geojson")

In [ ]:
# Read the bounding box and print
bbox = json.load(open("bbox.geojson", "r"))
bbox

In [14]:
# Establish Connection with eoAPI

api = Client.open("https://e011yr04a1.execute-api.us-west-2.amazonaws.com")
api.title

In [ ]:
# Define all related data.

EVENT_DETAILS = {
    'pakistan_flood': {
        'start_date': '2022-03-19T00:00:00Z',
        'end_date': '2023-02-18T00:00:00Z',
        'collections': ['HLSL30', 'HLSS30']
    },
    'mongolian_fire': {
        'start_date': '2022-04-19T00:00:00Z',
        'end_date': '2022-04-19T23:59:59Z',
        'collections': ['HLSL30']
    },
    'mississippi_tornado': {
        'start_date': '2023-02-19T00:00:00Z',
        'end_date': '2023-04-20T23:59:59Z',
        'collections': ['HLSL30', 'HLSS30']
    },
    'new_mexico_black_fire': {
        'start_date': '2020-05-16T00:00:00Z',
        'end_date': '2023-06-10T23:59:59Z',
        'collections': ['HLSL30', 'HLSS30']
    }
}

DERIVED_PRODUCTS = {
    "NDVI":{
        "HLSS30": ["B8A","B04"],
        "HLSL30": ["B05","B04"]
    },
    "EVI":{
        "HLSS30": ["B8A", "B04", "B02"],
        "HLSL30": ["B05", "B04", "B02"]
    },
    "SAVI":{
        "HLSS30": ["B8A", "B04"],
        "HLSL30": ["B05", "B04"]
    },
    "MSAVI":{
        "HLSS30": ["B8A", "B04"],
        "HLSL30": ["B05", "B04"]
    },
    "NDMI":{
        "HLSS30": ["B8A", "B11"],
        "HLSL30": ["B05", "B06"]
    },
    "NBR":{
        "HLSS30": ["B8A", "B12"],
        "HLSL30": ["B05", "B07"]
    },
    "NBR2":{
        "HLSS30": ["B11", "B12"],
        "HLSL30": ["B06", "B07"]
    },
    "NDWI":{
        "HLSS30": ["B03", "B8A"],
        "HLSL30": ["B03", "B05"]
    },
    "MNDWI1":{
        "HLSS30": ["B03", "B11"],
        "HLSL30": ["B03", "B06"]
    },
    "MNDWI2":{
        "HLSS30": ["B03", "B12"],
        "HLSL30": ["B03", "B07"]
    },
    "CL705":{
        "HLSS30": ["B8A", "B05"],
        "HLSL30": []
    },
    "TVI":{
        "HLSS30": ["B8A", "B03", "B04"],
        "HLSL30": ["B05", "B03", "B04"]
    },
}

In [ ]:
def preprocess(band_stack):
    """
    Preprocess provided band stack/rescale.
    """
    return band_stack.where(band_stack != -9999) * .0001

def calculate_NDVI(stack, band1_name, band2_name):
    """
    Calculate NDVI with NIR and RED bands:
    Args:
        band1_name: NIR band name for the stack
        band2_name: Red band name for the stack
    return:
        ndvi stack
    """
    band1, band2 = stack.sel(band=band1_name), stack.sel(band=band2_name)
    ndvi = (band1 - band2) / (band1 + band2)
    ndvi = ndvi.persist()
    return ndvi

def calculate_EVI(stack, band1_name, band2_name, band3_name):
    """
    Calculate ...
    Args:
        band1_name: NIR band name for the stack
        band2_name: Red band name for the stack
        band3_name: NIR band name for the stack
    return:
        EVI stack
    """
    band1, band2, band3 = stack.sel(band=band1_name), stack.sel(band=band2_name), stack.sel(band=band3_name)
    evi = 2.5 * ((band1 - band2) / (band1 + 6.0 * band2 - 7.5 * band3 + 1.0))
    evi = evi.persist()
    return evi

def calculate_SAVI(stack, band1_name, band2_name):
    """
    Calculate ...
    Args:
        band1_name: NIR band name for the stack
        band2_name: Red band name for the stack
    return:
        SAVI stack
    """
    band1, band2 = stack.sel(band=band1_name), stack.sel(band=band2_name)
    savi = ((band1 - band2) / (band1 + band2 + 0.5)) * 1.5
    savi = savi.persist()
    return savi

def calculate_MSAVI(stack, band1_name, band2_name):
    """
    Calculate ...
    Args:
        band1_name: NIR band name for the stack
        band2_name: Red band name for the stack
    return:
        MSAVI stack
    """
    band1, band2 = stack.sel(band=band1_name), stack.sel(band=band2_name)
    msavi = (2.0 * band1 + 1.0 - np.sqrt((2.0 * band1 + 1.0)**2.0 - 8.0 * (band1 - band2))) / 2.0
    msavi = msavi.persist()
    return msavi

def calculate_CL705(stack, band1_name, band2_name):
    """
    Calculate ...
    Args:
        band1_name: NIR band name for the stack
        band2_name: Red band name for the stack
    return:
        cl705 stack
    """
    band1, band2 = stack.sel(band=band1_name), stack.sel(band=band2_name)
    cl705 = (band1 / band2)**-1
    cl705 = cl705.persist()
    return cl705

def calculate_TVI(stack, band1_name, band2_name, band3_name):
    """
    Calculate ...
    Args:
        band1_name: NIR band name for the stack
        band2_name: Red band name for the stack
        band3_name: NIR band name for the stack
    return:
        TVI stack
    """
    band1, band2, band3 = stack.sel(band=band1_name), stack.sel(band=band2_name), stack.sel(band=band3_name)
    tvi = 0.5 * (120. * (band1 - band2) - 200. * (band3 - band2))
    tvi = tvi.persist()
    return tvi


In [ ]:
# Calculate NDVI for the selected location and date
event = 'pakistan_flood'

def prepare_stack(event, collection_name, bbox, start_date=None, end_date=None, ):
    """
    Prepare stack based on the selected event, datetime, and collection type
    ARGS: 
        event (str): Name of event, any key from EVENT_DETAILS
        collection_name (str): One of [HLSS30, HLSL30]
        bbox (list): List of bounding box coordinates (in geojson format).
        start_date (str): optional, start_date if custom date range
        start_date (str): optional, end_date if custom date range
    """
    if not(start_date) or not(end_date):
        start_date = EVENT_DETAILS[event]['start_date']
        end_date = EVENT_DETAILS[event]['end_date']
    results = api.search(
        intersects=bbox["features"][0]["geometry"],
        datetime=f"{start_date}/{end_date}",
        collections=[collection_name]
    )
    items = results.item_collection()
    for item in results.items():
        print(item.id)
    return items

    
# Prepare L30 Stack
l30_stack = prepare_stack(event, 'HLSL30')
l30_stack = preprocess(l30_stack)

# Prepare S30 Stack
s30_stack = prepare_stack(event, 'HLSS30')
s30_stack = preprocess(s30_stack)

# Calculate NDVI using L30
l30_ndvi = calculate_NDVI(l30_stack, *DERIVED_PRODUCTS['NDVI']['HLSL30'])

# Calculate NDVI using S30
s30_ndvi = calculate_NDVI(l30_stack, *DERIVED_PRODUCTS['NDVI']['HLSS30'])

# Prepare data for a day.
l30_ndvi_day = l30_ndvi.resample(time="1d").max()[0]
s30_ndvi_day = s30_ndvi.resample(time="1d").max()[0]

In [ ]:
# Visually see the NDVI using S30 for selected location, and date.
l30_ndvi_day.plot(vmin=-1, vmax=1, cmap="RdYlGn", figsize=(15,15))

In [ ]:
# Visually see the NDVI using S30 for selected location, and date.
s30_ndvi_day.plot(vmin=-1, vmax=1, cmap="RdYlGn", figsize=(15,15))